## 컨볼루션 신경망

1. DMLP
- 완전 연결 구조(Fully connected layer)로 높은 복잡도(파라미터수가 매우 많음)
- 학습이 매우 느리고, 과잉적합이 발생할 가능성이 있다.

2. 컨볼루션 신경망
- 격자 구조를 갖는 데이터에 적합
- 영상 분류나 문자 인식 등 인식 문제에 높은 성능
- 컨볼루션 연산을 수행하여 특징을 추출

### 컨볼루션 연산
- 해당하는 요소끼리만 곱해서 결과를 모두 더하는 선형 연산
- 입력으로부터 커널 영역 만큼만 선택되어 선택된 입력값과 커널 간 element-wise하게 곱해주고, 전부 sum한다.
- 커널은 일정 규칙(보폭)에 따라 움직이며 입력의 선택 영역을 계속 변경해준다.
- sum한 출력값들을 모아서 하나의 feature map을 생성한다.

### 보폭(Stride), 패딩(Padding)
- 보폭 : 커널을 다음 컨볼루션 연산을 위해 이동시키는 칸 수
- 패딩: 컨볼루션 결과의 크기를 조정하기 위해 입력 배열의 둘레를 확장하고 0으로 채우는 연산

### 풀링(Pooling)
- 일정 크기의 블록을 통합하여 하나의 대푯값으로 대체하는 연산
- 최대값 풀링(Max Pooling), 평균값 풀링(Average Pooling) 등 대푯값 선택 규칙을 별도로 선택한다

### 컨볼루션 신경망의 구조
- 특징 추출
    - 컨볼루션 연산을 수행하는 CONV 층
    - ReLU 연산을 수행하는 ReLU 층
    - 풀링 연산을 수행하는 Pool 층
- 추출된 특징을 분류나 회귀를 수행하는 다층 퍼셉트론
    - 전체 연결된(Fully Connected) FC층 반복
    - 분류의 경우 마지막 층에 소프트맥스(Softmax) 연산 수행
    
cf) Softmax, sigmoid -> 활성함수. Cross Entropy -> Loss function(for Softmax)

In [12]:
# AlexNet: 8개의 층, ReLU함수를 사용한 첫 모델. FC층에 Dropout 사용. Max Pooling 사용.

import torch
import torch.nn as nn
from torch.hub import load_state_dict_from_url
from typing import Any


__all__ = ['AlexNet', 'alexnet']


model_urls = {
    'alexnet': 'https://download.pytorch.org/models/alexnet-owt-4df8aa71.pth',
}


class AlexNet(nn.Module):

    def __init__(self, num_classes: int = 1000) -> None:
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            #Conv1
            # input: (N, Channel_in, Height, Width). 배치 사이즈, 채널의 수(depth), 데이터 높이, 데이터 너비
            # output: (N, Channel_out, Height_out, Width_out).
            # Conv2d의 stide, padding, dilation은 tuple이 들어올 수 있다. stride default = 1, padding = 0.
            #input channel : 3(RGB) , output channel : 64, kernel_size : 11 (11*11), stride : 4, padding : 2
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2), # output channel = output feature map 수. output featuremap size(H, W) = (W - F + 2P) / S + 1
            nn.ReLU(inplace=True), # inplace=True 하면, inplace 연산을 수행함, inplace 연산은 결과값을 새로운 변수에 값을 저장하는 대신 기존의 데이터를 대체하는것을 의미
            #Max Pool1
            nn.MaxPool2d(kernel_size=3, stride=2), # Pooling을 했어도 channel 수는 유지. input map의 size만 줄어든다(위 CONV output size)
            #Conv2
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            #Max Pool2
            nn.MaxPool2d(kernel_size=3, stride=2),
            #Conv3
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            ##Conv4
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #Conv5
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            #Max Pool3
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6)) # 어떤 input size가 들어오더라도 주어진 argument와 같은 사이즈의 output을 리턴한다((6,6))
        self.classifier = nn.Sequential(
            #드롭아웃 
            nn.Dropout(),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        #특징 추출 부분
        x = self.features(x)
        # Average Pooling
        x = self.avgpool(x)
        #output shape : (batch size * 256(channel), 6, 6)
        #Flatten
        x = torch.flatten(x, 1)
        #output shape (batch_size, 256 * 6* 6)
        #분류 분류 
        x = self.classifier(x)
        return x


def alexnet(pretrained: bool = False, progress: bool = True, **kwargs: Any) -> AlexNet:
    r"""AlexNet model architecture from the
    `"One weird trick..." <https://arxiv.org/abs/1404.5997>`_ paper.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
        progress (bool): If True, displays a progress bar of the download to stderr
    """
    model = AlexNet(**kwargs)
    if pretrained:
        state_dict = load_state_dict_from_url(model_urls['alexnet'],
                                              progress=progress)
        model.load_state_dict(state_dict)
    return model